In [36]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
#import lightgbm as lgb
#import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
#import shap  # package used to calculate Shap values
import time
import glob
import sys
import os
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression

print("Import finishes")

Import finishes


In [37]:
print('read csv...')
fulldf=pd.read_csv('full_DF.csv')
alltest=pd.read_csv('alltest.csv')
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
X_train=fulldf.iloc[:,1:]
y_train=fulldf.iloc[:,0]

#Scale train and test data
print('scale data...')
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(alltest), columns=alltest.columns)
print('end')
print(fulldf.shape)

read csv...
scale data...
end
(8252, 83)


In [34]:
print(X_train_scaled.head())
print(X_test_scaled.head())

   MEAN_full   SD_full   q0_full  q0.01_full  q0.05_full  q0.1_full  \
0   1.417288 -0.174309  0.196545    0.239325    0.090408   0.182526   
1   0.880347 -0.049123 -0.011966    0.166802    0.090408   0.182526   
2   0.798100 -0.000622 -0.011966    0.021756    0.090408   0.182526   
3   0.904591 -0.102634  0.133247    0.021756    0.090408   0.182526   
4   1.504410  0.043576  0.166758   -0.268336   -0.325657   0.182526   

   q0.25_full  q0.5_full  q0.75_full  q0.9_full  ...  SD_roll_sd_1500  \
0    1.597236   0.948201    0.265415   0.645392  ...        -0.187444   
1   -0.398281   0.948201    0.265415   0.645392  ...        -0.021650   
2   -0.398281   0.948201    0.265415   0.645392  ...         0.013566   
3   -0.398281   0.948201    0.265415   0.645392  ...        -0.133988   
4   -0.398281   0.948201    0.265415   0.645392  ...        -0.007133   

   q0_roll_sd_1500  q0.05_roll_sd_1500  q0.25_roll_sd_1500  \
0         0.661393            0.096891            0.096992   
1         

In [39]:
print('start')
print("RandomForest...")
rfc_model = RandomForestRegressor(random_state=0).fit(X_train_scaled,y_train)
submission['time_to_failure'] = rfc_model.predict(X_test_scaled)
submission.to_csv('submission_rf_new1.csv')
print('end')

start
RandomForest...
end


In [ ]:
print('start')
from sklearn.model_selection import GridSearchCV
'''
param_test1 = {'n_estimators':range(500,1001,100),
               'max_depth':range(2,5), 
               'min_samples_split':range(2,5),
               'min_samples_leaf':range(40,60,5),
               'max_features':range(5,15)}
'''
param_test1 = {"n_estimators": range(100,1001,100),
              "max_depth": range(2,5),
              "min_samples_split": range(2,5),
              "min_samples_leaf": [5, 10, 20],
              "max_leaf_nodes": [20, 40],
              "min_weight_fraction_leaf": [0.1]}

gsearch1 = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_test1, scoring=None,cv=5)
gsearch1.fit(X_train_scaled,y_train)
print(gsearch1.best_params_)
submission['time_to_failure'] = gsearch1.predict(X_test_scaled)
submission.to_csv('submission_rf_tune.csv')
print('end')

start
